## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

City_ID           City Country                 Date      Lat       Lng  \
0        0       Hermanus      ZA  2021-03-27 22:54:11 -34.4187   19.2345   
1        1  Batagay-Alyta      RU  2021-03-27 22:55:42  67.8006  130.4114   
2        2        Ahipara      NZ  2021-03-27 22:55:43 -35.1667  173.1667   
3        3  Marsh Harbour      BS  2021-03-27 22:55:43  26.5412  -77.0636   
4        4    Port Alfred      ZA  2021-03-27 22:54:11 -33.5906   26.8910   

   Max Temp  Humidity  Cloudiness  Wind Speed       Description  
0     57.99        79           5        1.01         clear sky  
1      3.78        89         100        3.15   overcast clouds  
2     66.16        90         100       12.03   overcast clouds  
3     76.26        71           0       14.09         clear sky  
4     63.36        74          49        8.86  scattered clouds

In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 72
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

City_ID                      City Country                 Date      Lat  \
3         3             Marsh Harbour      BS  2021-03-27 22:55:43  26.5412   
7         7                   Cayenne      GF  2021-03-27 22:55:44   4.9333   
17       17                   Durango      MX  2021-03-27 22:52:21  24.8333   
20       20                    Bonthe      SL  2021-03-27 22:55:49   7.5264   
21       21                      Fare      PF  2021-03-27 22:55:49 -16.7000   
24       24           Bambous Virieux      MU  2021-03-27 22:55:50 -20.3428   
27       27                    Maitum      PH  2021-03-27 22:55:51   6.0392   
34       34  Sao Gabriel Da Cachoeira      BR  2021-03-27 22:55:54  -0.1303   
36       36                   Rikitea      PF  2021-03-27 22:55:55 -23.1203   
37       37                     Vaini      TO  2021-03-27 22:55:56 -21.2000   

         Lng  Max Temp  Humidity  Cloudiness  Wind Speed       Description  
3   -77.0636     76.26        71           0       14.09         clear sky  
7   -52.3333     80.01        83          75        9.22     broken clouds  
17 -104.8333     77.41         8          60       22.70     broken clouds  
20  -12.5050     79.02        88           3        7.05         clear sky  
21 -151.0167     82.26        74          38       21.12  scattered clouds  
24   57.7575     77.00        88          75        5.75     broken clouds  
27  124.4986     79.00        83         100        2.21   overcast clouds  
34  -67.0892     74.23        96          72        3.00     moderate rain  
36 -134.9692     79.79        68           1       18.05         clear sky  
37 -175.2000     84.20        84          40        8.05  scattered clouds

In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        255
City           255
Country        255
Date           255
Lat            255
Lng            255
Max Temp       255
Humidity       255
Cloudiness     255
Wind Speed     255
Description    255
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

City Country  Max Temp       Description      Lat  \
3              Marsh Harbour      BS     76.26         clear sky  26.5412   
7                    Cayenne      GF     80.01     broken clouds   4.9333   
17                   Durango      MX     77.41     broken clouds  24.8333   
20                    Bonthe      SL     79.02         clear sky   7.5264   
21                      Fare      PF     82.26  scattered clouds -16.7000   
24           Bambous Virieux      MU     77.00     broken clouds -20.3428   
27                    Maitum      PH     79.00   overcast clouds   6.0392   
34  Sao Gabriel Da Cachoeira      BR     74.23     moderate rain  -0.1303   
36                   Rikitea      PF     79.79         clear sky -23.1203   
37                     Vaini      TO     84.20  scattered clouds -21.2000   

         Lng Hotel Name  
3   -77.0636             
7   -52.3333             
17 -104.8333             
20  -12.5050             
21 -151.0167             
24   57.7575             
27  124.4986             
34  -67.0892             
36 -134.9692             
37 -175.2000

In [12]:
# 6a. Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City           255
Country        255
Max Temp       255
Description    255
Lat            255
Lng            255
Hotel Name     255
dtype: int64

In [14]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))